<a href="https://colab.research.google.com/github/dhuanca/Pycon2022/blob/main/SegmentacionPyconBolivia2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 42.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c533785d5b1d9b3511d5fd3de0e25141996212a1f8041a296633a9d3e007eef9
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
spark = SparkSession.builder.appName("Segmentacion").getOrCreate()

In [6]:
df_telco = spark.read.csv('/content/churn_Operacion_2.csv', header=True, inferSchema=True)

In [7]:
df_telco.show(1)

+-----+--------+----------+-------------+-------------------+----------------+----------------------+------------------+----------------+-----------------+------------------+----------------+-----------------+--------------------+------------------+-------------------+-------------------+-----------------+------------------+------------------------------+-------+
|state| account| area_code| phone_number| international_plan| voice_mail_plan| number_vmail_messages| total_day_minutes| total_day_calls| total_day_charge| total_eve_minutes| total_eve_calls| total_eve_charge| total_night_minutes| total_night_calls| total_night_charge| total_intl_minutes| total_intl_calls| total_intl_charge| number_customer_service_calls| churn.|
+-----+--------+----------+-------------+-------------------+----------------+----------------------+------------------+----------------+-----------------+------------------+----------------+-----------------+--------------------+------------------+-------------------

In [8]:
df_telco.printSchema()

root
 |-- state: string (nullable = true)
 |--  account: double (nullable = true)
 |--  area_code: double (nullable = true)
 |--  phone_number: string (nullable = true)
 |--  international_plan: string (nullable = true)
 |--  voice_mail_plan: string (nullable = true)
 |--  number_vmail_messages: double (nullable = true)
 |--  total_day_minutes: double (nullable = true)
 |--  total_day_calls: double (nullable = true)
 |--  total_day_charge: double (nullable = true)
 |--  total_eve_minutes: double (nullable = true)
 |--  total_eve_calls: double (nullable = true)
 |--  total_eve_charge: double (nullable = true)
 |--  total_night_minutes: double (nullable = true)
 |--  total_night_calls: double (nullable = true)
 |--  total_night_charge: double (nullable = true)
 |--  total_intl_minutes: double (nullable = true)
 |--  total_intl_calls: double (nullable = true)
 |--  total_intl_charge: double (nullable = true)
 |--  number_customer_service_calls: double (nullable = true)
 |--  churn.: strin